# DS 320 Final Project - Thomas

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from tabulate import tabulate
from sklearn import metrics


In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


In [3]:
plays = pd.read_csv('/content/gdrive/MyDrive/plays.csv')
week1 = pd.read_csv('/content/gdrive/MyDrive/week1.csv')

In [4]:
week2 = pd.read_csv('/content/gdrive/MyDrive/football/week2.csv')
week3 = pd.read_csv('/content/gdrive/MyDrive/football/week3.csv')

In [5]:
week4 = pd.read_csv('/content/gdrive/MyDrive/football/week4.csv')
week5 = pd.read_csv('/content/gdrive/MyDrive/football/week5.csv')

In [6]:
week6 = pd.read_csv('/content/gdrive/MyDrive/football/week6.csv')
week7 = pd.read_csv('/content/gdrive/MyDrive/football/week7.csv')

In [7]:
week8 = pd.read_csv('/content/gdrive/MyDrive/football/week8.csv')
week9 = pd.read_csv('/content/gdrive/MyDrive/football/week9.csv')

In [8]:
week10 = pd.read_csv('/content/gdrive/MyDrive/football/week10.csv')
week11 = pd.read_csv('/content/gdrive/MyDrive/football/week11.csv')

In [9]:
week12 = pd.read_csv('/content/gdrive/MyDrive/football/week12.csv')
week13 = pd.read_csv('/content/gdrive/MyDrive/football/week13.csv')

In [10]:
week14 = pd.read_csv('/content/gdrive/MyDrive/football/week14.csv')
week15 = pd.read_csv('/content/gdrive/MyDrive/football/week15.csv')

In [11]:
week16 = pd.read_csv('/content/gdrive/MyDrive/football/week16.csv')
week17 = pd.read_csv('/content/gdrive/MyDrive/football/week17.csv')

In [12]:
players = pd.read_csv('/content/gdrive/MyDrive/players.csv')

In [13]:
df = plays.copy()
passes = df[df['playDescription'].str.contains(" pass ", case=False, na=False)]
passes.shape[0]

17851

In [14]:
words = "INTERCEPTED|PENALTY|FUMBLES|SACKED|INCOMPLETE"
passes["binaryPassResult"] = np.where(passes["playDescription"].str.contains(words, case=False, na=False), 0, 1)

<ipython-input-14-13eddcecc171>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes["binaryPassResult"] = np.where(passes["playDescription"].str.contains(words, case=False, na=False), 0, 1)


In [15]:
short = passes[passes['playDescription'].str.contains("short", case=False, na=False)]
deep = passes[passes['playDescription'].str.contains("deep", case=False, na=False)]

In [16]:
df = pd.concat([week1, week2, week3, week4, week5, week6, week7, week8, week9, week10, week11, week12, week13, week14, week15, week16, week17])

In [17]:
df = df[df["position"] == "WR"] 

In [18]:
df = df[df['event'] == 'pass_arrived']

In [19]:
df = df[['a', 's', 'dis','gameId', 'playId', 'nflId']]

In [20]:
df = df.merge(players, how = "left")

In [21]:
df = df[['a', 's', 'dis','gameId', 'playId', 'displayName']]

In [22]:
short = short[['gameId', 'playId', 'passResult', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter', 'playDescription']]

In [23]:
deep = deep[['gameId', 'playId', 'passResult', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter', 'playDescription']]

In [24]:
df_long = df.merge(deep, how = "inner")

In [25]:
df_short = df.merge(short, how = "inner")

In [26]:
df_short[['first', 'last']] = df_short['displayName'].str.split(' ', 1, expand=True)

In [27]:
df_long[['first', 'last']] = df_long['displayName'].str.split(' ', 1, expand=True)

In [28]:
df_short = df_short.dropna()
df_long = df_long.dropna()

In [29]:
df_short = df_short[df_short.apply(lambda x: x['last'] in x['playDescription'], axis = 1)]

In [30]:
df_long = df_long[df_long.apply(lambda x: x['last'] in x['playDescription'], axis = 1)]

In [31]:
df_short['binaryPassResult'] = [0 if x == 'I' else 1 for x in df_short["passResult"]]
df_long['binaryPassResult'] = [0 if x == 'I' else 1 for x in df_long["passResult"]]

In [32]:
df_long = df_long[['a', 's', 'dis','binaryPassResult', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter']]
df_short = df_short[['a', 's', 'dis','binaryPassResult', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter']]

In [33]:
X = df_short[['a', 's', 'dis', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter']]
y = df_short['binaryPassResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [34]:
short_clf=RandomForestClassifier(n_estimators=100)

short_clf.fit(X_train,y_train)
y_pred=short_clf.predict(X_test)


In [35]:
rfc_short_acc = metrics.accuracy_score(y_test, y_pred)

print("Model accuracy on short passes:",metrics.accuracy_score(y_test, y_pred))
print("Model auc on short passes:",metrics.roc_auc_score(y_test, y_pred))
print("Model rmse on short passes",metrics.mean_squared_error(y_test, y_pred))

Model accuracy on short passes: 0.8185443668993021
Model auc on short passes: 0.5131447867428822
Model rmse on short passes 0.1814556331006979


In [36]:
metrics.confusion_matrix(y_test, y_pred)

array([[  15,  337],
       [  27, 1627]])

In [37]:
X = df_long[['a', 's', 'dis', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter']]
y = df_long['binaryPassResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [38]:
long_clf=RandomForestClassifier(n_estimators=100)

long_clf.fit(X_train,y_train)
y_pred=long_clf.predict(X_test)

In [39]:
rfc_deep_acc = metrics.accuracy_score(y_test, y_pred)

print("Model accuracy on long passes:",metrics.accuracy_score(y_test, y_pred))
print("Model auc on short passes:",metrics.roc_auc_score(y_test, y_pred))
print("Model rmse on short passes",metrics.mean_squared_error(y_test, y_pred))

Model accuracy on long passes: 0.6350093109869647
Model auc on short passes: 0.5230392156862745
Model rmse on short passes 0.3649906890130354


In [40]:
from xgboost import XGBClassifier

X = df_short[['a', 's', 'dis', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter']]
y = df_short['binaryPassResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 7)

In [41]:
modelXGB = XGBClassifier(objective = 'binary:logistic', max_depth = 6, learning_rate = 0.03, n_estimators = 1000)
modelXGB.fit(X_train, y_train)

XGBClassifier(learning_rate=0.03, max_depth=6, n_estimators=1000)

In [42]:
y_pred = modelXGB.predict(X_test)
preds = [round(value) for value in y_pred]

accuracy_short = metrics.accuracy_score(y_test, preds)
print("Model accuracy on short passes:",metrics.accuracy_score(y_test, y_pred))
print("Model auc on short passes:",metrics.roc_auc_score(y_test, y_pred))
print("Model rmse on short passes",metrics.mean_squared_error(y_test, y_pred))

Model accuracy on short passes: 0.8095712861415753
Model auc on short passes: 0.5141504937010555
Model rmse on short passes 0.19042871385842472


In [43]:
metrics.confusion_matrix(y_test, y_pred)

array([[  20,  336],
       [  46, 1604]])

In [44]:

X = df_long[['a', 's', 'dis', 'absoluteYardlineNumber','yardsToGo', 'down', 'quarter']]
y = df_long['binaryPassResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 7)

In [45]:
model = XGBClassifier(objective = "binary:logistic", max_depth = 7, learning_rate = 0.05, n_estimators=10000 )
model.fit(X_train, y_train)

XGBClassifier(learning_rate=0.05, max_depth=7, n_estimators=10000)

In [46]:
y_pred = model.predict(X_test)

accuracy_deep = metrics.accuracy_score(y_test, y_pred)
print("Model accuracy on long passes:",metrics.accuracy_score(y_test, y_pred))
print("Model auc on short passes:",metrics.roc_auc_score(y_test, y_pred))
print("Model rmse on short passes",metrics.mean_squared_error(y_test, y_pred))

Model accuracy on long passes: 0.611731843575419
Model auc on short passes: 0.5193382792447523
Model rmse on short passes 0.388268156424581


In [47]:
metrics.confusion_matrix(y_test, y_pred)

array([[ 29,  90],
       [ 49, 190]])

In [48]:
accuracy_data = [["Random Forest Classifier", rfc_short_acc, rfc_deep_acc], ["XGBoost", accuracy_short, accuracy_deep]]

col_names = ["Model", "Short Passes", "Long Passes"]

print(tabulate(accuracy_data, headers=col_names))


Model                       Short Passes    Long Passes
------------------------  --------------  -------------
Random Forest Classifier        0.818544       0.635009
XGBoost                         0.809571       0.611732
